In [2]:
import networkx as nx
from collections import defaultdict
import numpy as np
import pickle
import collections
from tqdm import tqdm, tnrange

### [RQ1]
**Creating Graph**

In [3]:
DG = nx.DiGraph()

In [4]:
path = "C:/Users/guilh/Desktop/ADM/HW5/"

# i.e. our_edges = {node1:[node2,node4,node5], ..., 1:[2, 3, 5], 2:[5], 3:[6] ....}
# our_edges = defaultdict(list)

with open(path + "wiki-topcats-reduced.txt", "r") as f:
    #create graph
    for line in f.readlines():
        article1, article2 = line.split()
        DG.add_weighted_edges_from([(int(article1), int(article2), 1)])
#         our_edges[int(article1)].append(int(article2))

In [5]:
# Is graph directed
print("Is graph directed? ", nx.is_directed(DG), "\n") # check whether graph is directed or not

# example;
print("Since the neighbors of 52nd node are:", *list(DG.neighbors(52)),
      ",52nd node is not neighbor for 1069112nd node which has connection with", *list(DG.neighbors(1069112))," nodes. \n")

# The number of nodes
nodes_num = DG.number_of_nodes()
print("The number of nodes:", nodes_num, "\n") # also len(DG) works

# The number of edges
edges_num = DG.number_of_edges()
print("The number of edges:", edges_num)

Is graph directed?  True 

Since the neighbors of 52nd node are: 401135 1069112 1163551 ,52nd node is not neighbor for 1069112nd node which has connection with 1060396 1061304 1062611 1066969 1069008 1069113 1069258 1069275 1656982  nodes. 

The number of nodes: 461193 

The number of edges: 2645247


**Basic Information**:
+ If it is direct or not - **Direct**
+ The number of nodes
+ The number of edges
+ The average node degree. Is the graph dense?

**Graph density**
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges. The opposite, a graph with only a few edges, is a sparse graph. The distinction between sparse and dense graphs is rather vague, and depends on the context.

For directed graphs, the graph density is defined as:
$$D = \frac{|E|}{|V|(|V|-1)}$$

where E is the number of edges and V is the number of vertices in the graph. The maximum number of edges for an directed graph is $|V|(|V|-1).$

In [6]:
density = nodes_num / (edges_num*(edges_num-1))
print(density);

6.590986317548208e-08


### [RQ2] 
Given a category $C_0 = \{article_1, article_2, \dots \}$ as input we want to rank all of the nodes in V according to the block-ranking, where the blocks are represented by the categories:
$$block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}$$

Each category  corresponds to a list of nodes.

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

$$distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$$

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$. Moreover, the length of a path is given by the sum of the weights of the edges it is composed by.

##### Reading the file with categories

In [7]:
with open(path + "wiki-topcats-categories.txt", "r") as f2:
    categories = {} # {category0 : [article1, article2, ...], ...., 5: [23, 45, 6]}
    for cat_indx, line in enumerate(f2.readlines()):
        categories[cat_indx] = list(map(int, line.split(";")[1].split()))

#### Choosing categories which exist in our reduced graph:

In [8]:
len(DG.nodes)

461193

In [9]:
tmp_selected_category_indx = []
# filtering categories with nodes more than 3500
for i in range(len(categories)):
    if len(categories[i]) > 3500:
        tmp_selected_category_indx.append(i)

grouped_categories_nodes = [] # nodes grouped per category -- without C0 category
categories_nodes = set() # all nodes together -- without C0 category

# chose the category C0 with nodes only included in the DG graph:
C0 = set(categories[10839]).intersection(DG.nodes) #!!! Remember: we already have C0, do not create it later!

final_selected_category_indx = []
# chose categories with nodes only included in the DG graph:
for idx in tmp_selected_category_indx[1:]:
    tmp_categ = set(categories[idx]).intersection(DG.nodes)
    # if C_i contains different nodes than C0:
    C_i = tmp_categ - C0
    if len(C_i) != 0 and len(C_i) < 100000:
        final_selected_category_indx.append(idx)
        grouped_categories_nodes.append(C_i)
        categories_nodes = categories_nodes.union(C_i)

In [10]:
len(grouped_categories_nodes) + 1 #should be 34

33

In [11]:
len(C0)

3446

## BFS Shortest path

In [ ]:
def bfs_shortest_path(graph, start, categories_nodes):
    visited_dict = defaultdict(lambda:[False])
    queue = [start]
    visited_dict[start] = 0
    
    while queue:
        node = queue.pop(0)
        distance = visited_dict[node]
        try:
            for neighbour in graph.neighbors(node):
                if visited_dict[neighbour]==[False]:
                    visited_dict[neighbour] = distance + 1
                    queue.append(neighbour)
        except KeyError: pass
    return {node:visited_dict[node] for node in categories_nodes}

article_distances = {}
counter = 0

for idx, node in enumerate(C0):
    article_distances[node] = bfs_shortest_path(DG, node, categories_nodes)
    if (idx+1)%100==0:
        with open('distance_' + str(counter) + '.pkl', 'wb') as file:
            pickle.dump(article_distances, file, pickle.HIGHEST_PROTOCOL)
        article_distances = dict(); counter+=1

with open('distance_' + str(counter) + '.pkl', 'wb') as file:
    pickle.dump(article_distances, file, pickle.HIGHEST_PROTOCOL)

# Grouping categories with distances

In [12]:
path = "C:/Users/guilh/Desktop/ADM/HW5/distances_files/"

In [ ]:
for j in tnrange(35):
    
    with open(path + 'distance_' + str(j) + '.pkl', 'rb') as file:
        distance_dict = pickle.load(file)
    
    distances_categories = []
    
    #for each category selected previously excluding C0
    for i in final_selected_category_indx:
        distances = []
        #for each node in this selected category underanalysis
        for node in categories[i]:
            #for the starting node of C0 into our distance file
            for starting_node in distance_dict:
                #try to find the distances from C0 node to the node inside the category under analysis
                try:
                    d = distance_dict[starting_node][node]
                    if d != [False]:
                        distances.append(d)
                        #if distance is false append 9999
                    else: distances.append(9999)
                        
                except: pass
        #append the results of distances to this category
        distances_categories.append(distances)
    
    
    #save this distances and after each iteration we concatenate to it
    if j == 0:
        distances_concatenated = np.array([distances_categories])
    else:
        distances_concatenated = np.concatenate((distances_concatenated, np.array([distances_categories])), axis = 1)


In [13]:
distances_categories = {}
    
#for each category selected previously excluding C0
for i in tqdm(final_selected_category_indx):
    distances = []
    #for each node in this selected category underanalysis
    for j in tnrange(35):
        with open(path + 'distance_' + str(j) + '.pkl', 'rb') as file:
            distance_dict = pickle.load(file)
            for node in categories[i]:
                #for the starting node of C0 into our distance file
                for starting_node in distance_dict:
                    #try to find the distances from C0 node to the node inside the category under analysis
                    try:
                        d = distance_dict[starting_node][node]
                        if d != [False]:
                            distances.append(d)
                            #if distance is false append 9999
                        else: distances.append(9999)

                    except: pass
            #append the results of distances to this category
    distances = np.array(distances)
    median = np.median(distances)
    distances_categories[i] = median
    
    print(i, median)
    
    
#     #save this distances and after each iteration we concatenate to it
#     if j == 0:
#         distances_concatenated = np.array([distances_categories])
#     else:
#         distances_concatenated = np.concatenate((distances_concatenated, np.array([distances_categories])), axis = 1)


  0%|                                                                                           | 0/32 [00:00<?, ?it/s]


869 8.0


  3%|██▍                                                                            | 1/32 [18:04<9:20:15, 1084.37s/it]


876 10.0


  6%|█████                                                                           | 2/32 [29:24<8:01:32, 963.08s/it]

...

 94%|██████████████████████████████████████████████████████████████████████████     | 30/32 [5:44:33<22:05, 662.88s/it]


13996 7.0


 97%|████████████████████████████████████████████████████████████████████████████▌  | 31/32 [5:55:27<11:00, 660.29s/it]


16956 8.0


100%|███████████████████████████████████████████████████████████████████████████████| 32/32 [6:04:54<00:00, 632.28s/it]


In [15]:
distances_categories

{869: 8.0,
 876: 10.0,
 898: 9999.0,
 900: 9999.0,
 949: 9999.0,
 1881: 9999.0,
 2791: 7.0,
 4160: 8.0,
 5619: 6.0,
 6067: 7.0,
 6795: 9999.0,
 6893: 9.0,
 7628: 8.0,
 7889: 7.0,
 8732: 9999.0,
 8733: 9999.0,
 10062: 7.0,
 10139: 6.0,
 10251: 6.0,
 10555: 6.0,
 10564: 6.0,
 11640: 6.0,
 11641: 6.0,
 11648: 6.0,
 11649: 6.0,
 11992: 7.0,
 12631: 6.0,
 12941: 9999.0,
 12966: 8.0,
 13838: 6.0,
 13996: 7.0,
 16956: 8.0}

Sorting

In [16]:
sorted_by_value = sorted(distances_categories.items(), key=lambda kv: kv[1])
sorted_categories = [i[0] for i in sorted_by_value] 
sorted_categories

[5619,
 10139,
 10251,
 10555,
 10564,
 11640,
 11641,
 11648,
 11649,
 12631,
 13838,
 2791,
 6067,
 7889,
 10062,
 11992,
 13996,
 869,
 4160,
 7628,
 12966,
 16956,
 6893,
 876,
 898,
 900,
 949,
 1881,
 6795,
 8732,
 8733,
 12941]

In [14]:
distances_categories

{869: 8.0,
 876: 10.0,
 898: 9999.0,
 900: 9999.0,
 949: 9999.0,
 1881: 9999.0,
 2791: 7.0,
 4160: 8.0,
 5619: 6.0,
 6067: 7.0,
 6795: 9999.0,
 6893: 9.0,
 7628: 8.0,
 7889: 7.0,
 8732: 9999.0,
 8733: 9999.0,
 10062: 7.0,
 10139: 6.0,
 10251: 6.0,
 10555: 6.0,
 10564: 6.0,
 11640: 6.0,
 11641: 6.0,
 11648: 6.0,
 11649: 6.0,
 11992: 7.0,
 12631: 6.0,
 12941: 9999.0,
 12966: 8.0,
 13838: 6.0,
 13996: 7.0,
 16956: 8.0}

## Block Ranging Algorithm - Step 1 , 2 , 3

## Step 1

In [ ]:
C0 = categories[10839]
induced_graph = DG.subgraph(C0) # create sub graph for only category zero

In [ ]:
def C0_sum_weights_inedges(induced_graph):
    # Iterate to get sum of weights of in-edges
    all_weights = {} # it will look like {node1:sum_of_weights, node2:sum_of_weights, ...}
    for (node1,node2,data) in induced_graph.edges(data=True):
        if node2 not in all_weights.keys(): # we consider node2 because we're checking how many "incoming" neighboors
            all_weights[node2] = data['weight'] # if node2 doesn't exist in all_weights, just add initial weight
        else:
            all_weights[node2] += data['weight'] # if node2 already exists in all_weights, add weight up like cumulate weight
    
    # if there is no incoming neigboors, detect these nodes and give their values as zero
    for zero_node in list(set(induced_graph.nodes()) - set(all_weights.keys())):
        all_weights[zero_node]=0
        
    all_weights = sorted(all_weights.items(), key=lambda kv: kv[1], reverse=True) # sort with descending order by values in dictionary 

    return all_weights

In [ ]:
C0_score = C0_sum_weights_inedges(induced_graph)
C0_score # score for just category 0

## Step 2

In [ ]:
C1 = categories[2] 
sub_graph = G.subgraph(list(C0) + C1) # create sub graph for only category 0 and category 1

In [ ]:
C1_score = {} # it is the score for only category 1

""" apply same steps as step 1. The only difference is we use C0 as induced category. That means Since C0 is the first 
    ordered category, we need to take into account if there incoming arrows(directions) from catefory 0. We have to count
    also. So thats why we create sub graph with C0 and C1"""
for (node1,node2,data) in sub_graph.edges(data=True):
    if node2 in C1:
        if node2 not in C1_score.keys():
            C1_score[node2] = data['weight']
        else:
            C1_score[node2] += data['weight']
            
for zero_node in list(set(C1) - set(C1_score.keys())):
    C1_score[zero_node]=0
        
C1_score = sorted(C1_score.items(), key=lambda kv: kv[1], reverse=True) # sort by values
C1_score # for only category 1

## Step 3


In [ ]:
def score(DG, ordered_category_indx, C0_score):
    C_0 = list(categories[ordered_category_indx[0]])
    cum_nodes_list = C_0 # cumulative nodes list
    all_weights = {}
    all_weights.update(C0_score) # append induced category rank and sum of weights

    for cat_idx in ordered_category_indx:
        C_i = list(categories[cat_idx])
        cum_nodes_list = cum_nodes_list + C_i # we need to build sub graph cumulatively
        sub_graph = DG.subgraph(cum_nodes_list)
        
        cat_weights = {} # weights for only category C_i
        for (node1,node2,data) in sub_graph.edges(data=True):
            if node2 in C_i:
                if node2 not in cat_weights.keys():
                    cat_weights[node2] = data['weight']
                else:
                    cat_weights[node2] += data['weight']
    
        for zero_node in list(set(C_i) - set(cat_weights.keys())):
            cat_weights[zero_node] = 0
            
        cat_weights = sorted(cat_weights.items(), key=lambda kv: kv[1], reverse=True) # before adding to all_weights sort weights just inside the category C_i
        all_weights.update(cat_weights) # add cat_weights to all_weights
    
    return all_weights

In [ ]:
import time
start_time = time.time()

ordered_category_indx = [1,2,3] # in hw example order: C0,C1,C2
nodes_rank = score(G, ordered_category_indx, C0_score)
print("nodes rank and their weights:", nodes_rank)
print("nodes rank:", nodes_rank.keys())

delta = time.time() - start_time
print("running time:", delta)

In [ ]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
B = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [ ]:
np.concatenate((A,B), axis=1)

In [ ]:
A